# **Object Detection - Models Training**

---

In [1]:
from collections import Counter
from glob import glob
import json
import os
import pathlib
from PIL import Image
import random
from rfdetr import RFDETRBase
import shutil
import torch
from tqdm import tqdm
from ultralytics import YOLO

In [2]:
def get_image_label_pairs(
    image_root, annotation_root, image_ext="jpg", label_ext="txt"
):
    pairs = []
    image_files = glob(os.path.join(image_root, "**", f"*.{image_ext}"), recursive=True)

    for img in image_files:
        rel_path = os.path.relpath(img, image_root)
        label = os.path.join(
            annotation_root, os.path.splitext(rel_path)[0] + f".{label_ext}"
        )

        if os.path.exists(label):
            pairs.append((img, label))
        else:
            print(f"Warning: Label file not found for image {img}")

    return pairs

In [3]:
image_root = "../../data/augmented_data/frames"
annotation_root = "../../data/annotations"
pairs = get_image_label_pairs(image_root, annotation_root)
print(f"Found {len(pairs)} image-label pairs.")

Found 19122 image-label pairs.


In [4]:
empty_files = []

for root, _, files in os.walk(annotation_root):
    for file in files:
        file_path = os.path.join(root, file)

        if os.stat(file_path).st_size == 0:
            empty_files.append(file_path)

print(f"Found {len(empty_files)} empty label files.")

Found 0 empty label files.


In [5]:
label_counts = Counter()

for root, _, files in os.walk(annotation_root):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            lines = f.readlines()

            for line in lines:
                class_id = line.split()[0]
                label_counts[class_id] += 1

label_counts

Counter({'0': 31261, '1': 30493, '4': 30446, '2': 29889, '3': 26814})

In [6]:
# Paths
image_dir = "../../data/augmented_data/frames"
label_dir = "../../data/annotations"
output_dir = "../../data/split_data"

# Create output folders
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

# Get all images
all_images = []
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            all_images.append(os.path.join(root, file))

# Shuffle and split
random.shuffle(all_images)
train_ratio = 0.7
val_ratio = 0.2
train_count = int(len(all_images) * train_ratio)
val_count = int(len(all_images) * val_ratio)

train_images = all_images[:train_count]
val_images = all_images[train_count : train_count + val_count]
test_images = all_images[train_count + val_count :]

# Move files
def move_files(image_list, split):
    for img_path in image_list:
        # Get label path
        rel_path = os.path.relpath(img_path, image_dir)
        label_path = os.path.join(label_dir, os.path.splitext(rel_path)[0] + ".txt")

        # Define new paths
        new_img_path = os.path.join(
            output_dir, "images", split, os.path.basename(img_path)
        )
        new_label_path = os.path.join(
            output_dir, "labels", split, os.path.basename(label_path)
        )

        # Copy files
        shutil.copy(img_path, new_img_path)
        if os.path.exists(label_path):
            shutil.copy(label_path, new_label_path)


# Move train, val & test files
move_files(train_images, "train")
move_files(val_images, "val")
move_files(test_images, "test")

print(
    f"Dataset split completed: {len(train_images)} train, {len(val_images)} val, {len(test_images)} test"
)

Dataset split completed: 13385 train, 3824 val, 1913 test


In [7]:
yolo = YOLO(verbose=False)

results = yolo.train(
    data="../../data/dataset.yaml",
    epochs=100,
    batch=4,
    device="cuda" if torch.cuda.is_available() else "cpu",
    multi_scale=True,
    patience=5,
    lr0=1e-4,
    augment=True,
)

100%|██████████| 5.35M/5.35M [00:00<00:00, 25.5MB/s]


New https://pypi.org/project/ultralytics/8.3.140 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.133  Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../data/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=True, name=train, nbs=64, nms=False, opset=None,

train: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\train... 2097 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2097/2097 [00:02<00:00, 702.63it/s]


train: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 32.135.8 MB/s, size: 226.1 KB)


val: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\val... 1795 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1795/1795 [00:02<00:00, 771.71it/s]

...

     31/100      2.05G     0.7995     0.4486     0.8114         15        896: 100%|██████████| 525/525 [00:58<00:00,  8.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 225/225 [00:17<00:00, 13.07it/s]

                   all       1795      14205       0.99      0.976      0.991      0.845
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 26, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



31 epochs completed in 0.669 hours.
Optimizer stripped from C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train\weights\best.pt, 5.5MB

Validating C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train\weights\best.pt...
Ultralytics 8.3.133  Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 225/225 [00:24<00:00,  9.11it/s]


                   all       1795      14205      0.984      0.974      0.991      0.799
                  blue       1516       3125      0.981      0.961      0.979        0.8
                 green       1506       2950      0.985      0.964      0.992      0.801
                orange       1484       2863      0.984      0.987      0.994      0.809
                   red       1227       2397      0.985      0.985      0.994      0.804
                yellow       1447       2870      0.986      0.975      0.994      0.781
Speed: 0.2ms preprocess, 6.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\runs\detect\train


In [9]:
def yolo_to_coco_bbox(bbox, img_w, img_h):
    # YOLO: [x_center, y_center, width, height] normalized
    # COCO: [x_min, y_min, width, height] absolute
    x_c, y_c, w, h = bbox
    x_c *= img_w
    y_c *= img_h
    w *= img_w
    h *= img_h
    x_min = x_c - w / 2
    y_min = y_c - h / 2
    return [x_min, y_min, w, h]

def convert_split_to_coco(split_dir, split):
    images_dir = split_dir / "images" / split
    labels_dir = split_dir / "labels" / split
    image_files = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.png"))
    classes = {
        "blue": 0,
        "green": 1,
        "orange": 2,
        "red": 3,
        "yellow": 4,
    }
    categories = [
        {"id": cid, "name": name, "supercategory": "none"}
        for name, cid in classes.items()
    ]
    images = []
    annotations = []
    ann_id = 1

    for img_id, img_path in enumerate(tqdm(image_files, desc=f"Processing {split}")):
        try:
            with Image.open(img_path) as im:
                width, height = im.size
        except Exception as e:
            print(f"Could not open {img_path}: {e}")
            continue

        images.append({
            "id": img_id,
            "file_name": f"../../split_data/images/{split}/{img_path.name}",
            "width": width,
            "height": height,
        })

        label_path = labels_dir / (img_path.stem + ".txt")
        if not label_path.exists():
            continue

        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                class_id = int(float(parts[0]))
                bbox = list(map(float, parts[1:5]))
                coco_bbox = yolo_to_coco_bbox(bbox, width, height)
                annotations.append({
                    "id": ann_id,
                    "image_id": img_id,
                    "category_id": class_id,
                    "bbox": coco_bbox,
                    "area": coco_bbox[2] * coco_bbox[3],
                    "iscrowd": 0,
                    "segmentation": [],
                })
                ann_id += 1

    coco_dict = {
        "images": images,
        "annotations": annotations,
        "categories": categories,
    }
    return coco_dict

split_dir = pathlib.Path("../../data/split_data")
output_base = pathlib.Path("../../data/coco_like")
output_base.mkdir(exist_ok=True)

for split in ["train", "valid", "test"]:
    coco_dict = convert_split_to_coco(split_dir, split)
    out_dir = output_base / split
    out_dir.mkdir(parents=True, exist_ok=True)
    with open(out_dir / "_annotations.coco.json", "w") as f:
        json.dump(coco_dict, f, indent=2)

Processing train: 100%|██████████| 2097/2097 [00:00<00:00, 2167.07it/s]
Processing valid: 0it [00:00, ?it/s]
Processing test: 100%|██████████| 1673/1673 [00:13<00:00, 127.38it/s]


In [6]:
rfdetr = RFDETRBase()

rfdetr.train(
    dataset_dir="../../data/coco_like",
    epochs=10,
    batch_size=4,
    grad_accum_steps=4,
    lr=1e-4,
    early_stopping=True,
    early_stopping_patience=5,
    resume="./checkpoint.pth",
)

Loading pretrain weights


num_classes mismatch: model has 90 classes, but your dataset has 5 classes
reinitializing your detection head with 5 classes.


FailedPreconditionError: output is not a directory